> library dependencies

+ itunes API searcher
    + ```$ pip install pyitunes```

In [21]:
import itunes
import pandas as pd 
import matplotlib.pyplot as plt
import bs4 as bs
import urllib.request

### this is a wrapper for the [iTunes Search Api]()

In [22]:
# for example
item = itunes.search (query='star wars')
print (item)

[<Feature-Movie>: Star Wars: The Force Awakens, <Feature-Movie>: Star Wars: The Phantom Menace, <Feature-Movie>: Star Wars: The Empire Strikes Back, <Tv-Episode>: R2 Come Home, <Feature-Movie>: Star Wars: Return of the Jedi, <Feature-Movie>: Star Wars: A New Hope, <Feature-Movie>: Star Wars: Attack of the Clones, <Feature-Movie>: Star Wars: Revenge of the Sith, <Podcast>: Rebel Force Radio: Star Wars Podcast, <Tv-Episode>: Bombad Jedi, <Tv-Episode>: An Old Friend, <Song>: Star Wars, <Tv-Episode>: Weapons Factory, <Tv-Episode>: The Lawless, <Tv-Episode>: Citadel Rescue, <Tv-Episode>: Point of No Return, <Tv-Episode>: Bounty, <Tv-Episode>: Ambush, <Tv-Episode>: The Box, <Tv-Episode>: Rising Malevolence, <Tv-Episode>: Rookies, <Tv-Episode>: The Unknown, <Tv-Episode>: Defenders of Peace, <Tv-Episode>: Downfall of a Droid, <Tv-Episode>: Sacrifice, <Tv-Episode>: Shadow of Malevolence, <Tv-Episode>: Hostage Crisis, <Tv-Episode>: Mystery of the Thousand Moons, <Tv-Episode>: Conspiracy]


## This searches for podcasts, instead of music albums,
### and prroduces

### Set the query term and Run All Cells
### ⇩


In [23]:
# queryterm = 'cystic fibrosis'
queryterm = 'infectious disease'
# queryterm = 'genetic'
# queryterm = 'covid19'
# queryterm = 'foreclosure'
podsearch = itunes.search(query=queryterm, media='podcast')

In [24]:
for p in podsearch:
    print (p)

<Podcast>: Infectious Diseases Society of America
<Podcast>: Persiflagers Infectious Disease Puscast
<Podcast>: Emerging Infectious Diseases
<Podcast>: Persiflagers Infectious Disease Puscast
<Podcast>: Infectious Disease
<Podcast>: Breakpoints
<Podcast>: The Open Forum Infectious Diseases Podcast
<Podcast>: Infectious Disease & Antibiotic Resistance
<Podcast>: PeerView Infectious Diseases CME/CNE/CPE Audio Podcast
<Podcast>: Infectious Disease Dynamics
<Podcast>: Disease State - Infectious
<Podcast>: PeerVoice Immunology & Infectious Disease Video
<Podcast>: Infectious Questions : An Infectious Diseases Public Health Podcast
<Podcast>: Infectious Diseases
<Podcast>: Podcasts – PKIDs Blog
<Podcast>: Infectious Diseases and Pandemics (audio)
<Podcast>: Infectious Diseases and Pandemics (video)
<Podcast>: PeerVoice Immunology & Infectious Disease Audio
<Podcast>: PeerView Infectious Diseases CME/CNE/CPE Video Podcast
<Podcast>: The History of Infectious Diseases


In [25]:
# let's looks at what gets returned for each item in that list of Podcast search results
print (vars(podsearch[0]))

{'id': 519582740, 'name': 'Infectious Diseases Society of America', 'url': 'https://podcasts.apple.com/us/podcast/infectious-diseases-society-of-america/id519582740?uo=4', '_release_date': None, 'artwork': {'30': 'https://is3-ssl.mzstatic.com/image/thumb/Podcasts6/v4/97/ac/53/97ac535a-8e19-5a04-df2c-a5713369e99d/mza_158942553313998590.jpg/30x30bb.jpg', '60': 'https://is3-ssl.mzstatic.com/image/thumb/Podcasts6/v4/97/ac/53/97ac535a-8e19-5a04-df2c-a5713369e99d/mza_158942553313998590.jpg/60x60bb.jpg', '600': 'https://is3-ssl.mzstatic.com/image/thumb/Podcasts6/v4/97/ac/53/97ac535a-8e19-5a04-df2c-a5713369e99d/mza_158942553313998590.jpg/600x600bb.jpg'}, 'json': {'wrapperType': 'track', 'kind': 'podcast', 'collectionId': 519582740, 'trackId': 519582740, 'artistName': 'IDSA', 'collectionName': 'Infectious Diseases Society of America', 'trackName': 'Infectious Diseases Society of America', 'collectionCensoredName': 'Infectious Diseases Society of America', 'trackCensoredName': 'Infectious Diseas

### ⇪ A little wierd, looks like most of what we want is packed into
### a "JSON" attribute, so...there it is.

- - -

# So let's lookup some stuff about the podcast in question

+ no. of individual posts / episodes per year since publishing
+ length posts (minutes)
+ rating per channel +/or track, if available

### first for # posts / episodes / media URL

In [26]:
print (f'{len(podsearch)} channels\n')
for channel in podsearch:
    print (f'{channel.json["trackCount"]} podcasts in "{channel.json["collectionName"]}" (with id = {channel.id})')
    print (f'The last post in this channel was {channel.json["releaseDate"]}, and iTunes page lising all podcast episodes:')
    print (f'{channel.json["collectionViewUrl"]}\n')

20 channels

54 podcasts in "Infectious Diseases Society of America" (with id = 519582740)
The last post in this channel was 2020-05-23T07:30:00Z, and iTunes page lising all podcast episodes:
https://podcasts.apple.com/us/podcast/infectious-diseases-society-of-america/id519582740?uo=4

300 podcasts in "Persiflagers Infectious Disease Puscast" (with id = 79930757)
The last post in this channel was 2020-05-17T08:25:00Z, and iTunes page lising all podcast episodes:
https://podcasts.apple.com/us/podcast/persiflagers-infectious-disease-puscast/id79930757?uo=4

50 podcasts in "Emerging Infectious Diseases" (with id = 212828612)
The last post in this channel was 2020-05-21T15:56:00Z, and iTunes page lising all podcast episodes:
https://podcasts.apple.com/us/podcast/emerging-infectious-diseases/id212828612?uo=4

300 podcasts in "Persiflagers Infectious Disease Puscast" (with id = 1153935086)
The last post in this channel was 2020-05-17T08:25:00Z, and iTunes page lising all podcast episodes:
ht

## Fine.
### Uinfortunately, that 'iTunes Collection Preview' page only lists the first few episodes for the channel
### But podcast publisher's feed (not Apple) includes ALL podcasts 

In [27]:
channel_number = []
channel_name = []
channel_episodecount = []
channel_itunespreview = []
channel_feed = []

In [28]:
# "feedUrl" is not consistenly present, so handle missing Attributes

# definition to make write podcast channels to an array
def writeKeys(channel):
    channel_number.append(channel.id)
    channel_name.append(channel.json["collectionName"])
    channel_episodecount.append(channel.json["trackCount"])
    channel_itunespreview.append(channel.json["collectionViewUrl"])
    # feedURL is not consistent, check if present
    try:
       channel_feed.append(channel.json["feedUrl"])
    except KeyError as e:
        print (f'{e} does not exist in this channel')
        channel_feed.append(None)

In [29]:
for channel in podsearch:
    writeKeys(channel)

'feedUrl' does not exist in this channel
'feedUrl' does not exist in this channel
'feedUrl' does not exist in this channel
'feedUrl' does not exist in this channel


In [30]:
len(channel_name)

20

In [31]:
# create an "Channels" dataframe
data = {'ID': channel_number,
        'CHANNEL_NAME': channel_name,
        'EPISODE_COUNT': channel_episodecount,
        'ITUNES_PREVIEW_URL': channel_itunespreview,
        'PUBLISHER_FEED': channel_feed
       }
channels = pd.DataFrame(data,columns=['ID','CHANNEL_NAME','EPISODE_COUNT','ITUNES_PREVIEW_URL','PUBLISHER_FEED'])
channels

,ID,CHANNEL_NAME,EPISODE_COUNT,ITUNES_PREVIEW_URL,PUBLISHER_FEED
0,519582740,Infectious Diseases Society of America,54,https://podcasts.apple.com/us/podcast/infectio...,https://idsocietypodcast.libsyn.com/rss
1,79930757,Persiflagers Infectious Disease Puscast,300,https://podcasts.apple.com/us/podcast/persifla...,http://www.pusware.com/podcast.xml
2,212828612,Emerging Infectious Diseases,50,https://podcasts.apple.com/us/podcast/emerging...,https://www2c.cdc.gov/podcasts/createrss.asp?t...
3,1153935086,Persiflagers Infectious Disease Puscast,300,https://podcasts.apple.com/us/podcast/persifla...,http://www.pusware.com/podcast.xml
4,415333788,Infectious Disease,49,https://podcasts.apple.com/us/podcast/infectio...,http://wwwf.imperial.ac.uk/imedia/itunes_colle...
5,1470308447,Breakpoints,23,https://podcasts.apple.com/us/podcast/breakpoi...,https://pinecast.com/feed/sidp
6,1368469324,The Open Forum Infectious Diseases Podcast,28,https://podcasts.apple.com/us/podcast/the-open...,https://feeds.blubrry.com/feeds/the_ofid.xml
7,621579678,Infectious Disease & Antibiotic Resistance,7,https://podcasts.apple.com/us/podcast/infectio...,None
8,275853748,PeerView Infectious Diseases CME/CNE/CPE Audio...,16,https://podcasts.apple.com/us/podcast/peerview...,https://c.peerview.com/podcast/xml/infectiousd...
9,833385368,Infectious Disease Dynamics,53,https://podcasts.apple.com/us/podcast/infectio...,http://rss.sms.cam.ac.uk/rss/collection/153939...


In [32]:
channels.PUBLISHER_FEED[0]

'https://idsocietypodcast.libsyn.com/rss'

## Next: create an "Episodes" dataframe
### Constructed by keys in this 'dict'-like feed of each individual podcast episode

In [44]:
# what are the fields?
n = 6
request = urllib.request.Request(channels.PUBLISHER_FEED[n],headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36'})
source = urllib.request.urlopen(request)

# this is an XML feel
# inspection reveals that each podcast is encapsulated in <item> tags
soup = bs.BeautifulSoup(source,'lxml')
rows = soup.find_all('item')
print (f'"{channels.CHANNEL_NAME[n]}" channel has {len(rows)} episodes: here is the {n}th episode from that feed:\n')
print (rows[n])

"The Open Forum Infectious Diseases Podcast" channel has 28 episodes: here is the 6th episode from that feed:

<item>
<title>An interview with Dr. Eric Winer</title>
<link/>https://www.blubrry.com/the_ofid/44509529/an-interview-with-dr-eric-winer/
      <guid>http://www.blubrry.com/the_ofid/44509529/an-interview-with-dr-eric-winer/</guid>
<dc:creator>Oxford University Press</dc:creator>
<pubdate>Mon, 12 Nov 2018 08:24:00 -0500</pubdate>
<description>&lt;p&gt;In this episode, OFID Editor Paul Sax, MD, interviews Eric Winer, MD, about his enduring battle with hemophilia. Diagnosed at 13 months old, he shares how early exposure to hospitals and doctors ignited his drive to become a physician, and how his subsequent diagnoses with HIV, hepatitis C, and numerous treatment complications did not deter his work resolve or his determination to have a family – and an extraordinarily productive career.&lt;/p&gt;</description>
<content:encoded><span>In this episode, </span><em>OFID</em><span> Edit

In [34]:
for r in rows:
    per = r.find_all("itunes:duration")
    print (per)

[<itunes:duration>00:33:28</itunes:duration>]
[<itunes:duration>00:37:04</itunes:duration>]
[<itunes:duration>01:01:09</itunes:duration>]
[<itunes:duration>01:01:38</itunes:duration>]
[<itunes:duration>01:24:14</itunes:duration>]
[<itunes:duration>00:46:56</itunes:duration>]
[<itunes:duration>01:15:18</itunes:duration>]
[<itunes:duration>00:48:50</itunes:duration>]
[<itunes:duration>00:53:57</itunes:duration>]
[<itunes:duration>01:15:42</itunes:duration>]
[<itunes:duration>01:05:11</itunes:duration>]
[<itunes:duration>00:57:30</itunes:duration>]
[<itunes:duration>00:52:19</itunes:duration>]
[<itunes:duration>00:31:51</itunes:duration>]
[<itunes:duration>00:35:56</itunes:duration>]
[<itunes:duration>00:34:49</itunes:duration>]
[<itunes:duration>00:35:51</itunes:duration>]
[<itunes:duration>00:30:13</itunes:duration>]
[<itunes:duration>00:29:47</itunes:duration>]
[<itunes:duration>00:31:19</itunes:duration>]
[<itunes:duration>00:27:39</itunes:duration>]
[<itunes:duration>00:23:26</itunes

In [35]:
# df = pd.DataFrame(columns = ['ID','CHANNEL_NAME','EPISODE_TITLE','DATE_PUBLISHED','EPISODE_LENGTH','MEDIA_URL'])

In [36]:
channel_ids = []
channel_names = []
episode_titles = []
episode_published = []
episode_length = []
episode_mediaurls = []

In [37]:
def getEpisodeList(url):
    #since it's a request from the messy web, let's add some error handling
    try:
        request = urllib.request.Request(url,headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36'})
        source = urllib.request.urlopen(request)
    except urllib.error.HTTPError as e:
        print (f'Failed {e}: {url}')
        return None
    try:
        soup = bs.BeautifulSoup(source,'lxml')
        episodes = soup.find_all('item')
    except AttributeError as e:
        return None
    return episodes

In [38]:
def writeEpisode(row):
    if (row["PUBLISHER_FEED"]!=None):
#         print(row["ID"])
        #this is where you extract and append the individual Episode keys
        #repeating these two lines for each episode added to array
        r = getEpisodeList(row["PUBLISHER_FEED"])
        print (f'This podcast has {len(r)} episodes')
        for x in r:
            episode_titles.append(x.text)
            channel_ids.append(row["ID"])
            channel_names.append(row["CHANNEL_NAME"])
            try:
                length = x.find("itunes:duration")
                episode_length.append(length.text)
            except:
                print ("Nope, no <itunes:duration>")

#                 published = x.pubdate
#                 episode_published.append(published.text)                
channels.apply(writeEpisode, axis = 1)

This podcast has 54 episodes
This podcast has 323 episodes
Nope, no <itunes:duration>
This podcast has 50 episodes
This podcast has 323 episodes
Nope, no <itunes:duration>
This podcast has 49 episodes
This podcast has 23 episodes
This podcast has 28 episodes
Nope, no <itunes:duration>
Nope, no <itunes:duration>
Nope, no <itunes:duration>
Nope, no <itunes:duration>
Nope, no <itunes:duration>
Nope, no <itunes:duration>
Nope, no <itunes:duration>
Nope, no <itunes:duration>
Nope, no <itunes:duration>
Nope, no <itunes:duration>
Nope, no <itunes:duration>
Nope, no <itunes:duration>
Nope, no <itunes:duration>
Nope, no <itunes:duration>
Nope, no <itunes:duration>
Nope, no <itunes:duration>
Nope, no <itunes:duration>
Nope, no <itunes:duration>
Nope, no <itunes:duration>
Nope, no <itunes:duration>
Nope, no <itunes:duration>
Nope, no <itunes:duration>
Nope, no <itunes:duration>
Nope, no <itunes:duration>
Nope, no <itunes:duration>
Nope, no <itunes:duration>
Nope, no <itunes:duration>
Nope, no <it

0     None
1     None
2     None
3     None
4     None
5     None
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18    None
19    None
dtype: object

In [39]:
data = {'ID': channel_ids,
        'CHANNEL_NAME': channel_names,
        'EPISODE_TITLE': episode_titles
       }
episodes = pd.DataFrame(data, columns =['ID','CHANNEL_NAME','EPISODE_TITLE'])
episodes

,ID,CHANNEL_NAME,EPISODE_TITLE
0,519582740,Infectious Diseases Society of America,"\nCoronavirus: Educating ID Learners (May 23, ..."
1,519582740,Infectious Diseases Society of America,\nCoronavirus: Strategies for Safely Reopening...
2,519582740,Infectious Diseases Society of America,"\nCoronavirus: Secondary Infections (May 16, 2..."
3,519582740,Infectious Diseases Society of America,"\nCoronavirus: The Disparate Impact (May 12, 2..."
4,519582740,Infectious Diseases Society of America,"\nCoronavirus: Diagnostic Guidelines (May 9, 2..."
...,...,...,...
1040,288732495,PeerView Infectious Diseases CME/CNE/CPE Video...,"\nRoy F. Chemaly, MD, MPH, FIDSA, FACP - Manag..."
1041,288732495,PeerView Infectious Diseases CME/CNE/CPE Video...,"\nRichard S. Finn, MD - Breaking the Paradox: ..."
1042,288732495,PeerView Infectious Diseases CME/CNE/CPE Video...,"\nGregory Piazza, MD, MS - Turning the Tide on..."
1043,288732495,PeerView Infectious Diseases CME/CNE/CPE Video...,"\nMark S. Sulkowski, MD - Addressing the Evolv..."
